Built upon previous work [05212023 Country Names Record Linkage Part I (cosine-similarity).ipynb](https://github.com/tiangenglu/recordlinkage/blob/main/05212023%20Country%20Names%20Record%20Linkage%20Part%20I%20\(cosine-similarity\).ipynb)

In [1]:
import os
import numpy as np
import pandas as pd
import boto3
import json
import io
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML

# Access data from S3

In [2]:
with open("aws_credential.txt", 'r') as file:
    aws_credential=json.load(file)
s3=boto3.Session(
    profile_name = None, 
    region_name = 'us-east-2').client(
    's3',
    aws_access_key_id=aws_credential['access_key'],
    aws_secret_access_key=aws_credential['secret_key'])

## Country codes `.txt` to `bytes` to `pd.DataFrame`

In [4]:
content_list1=s3.list_objects(Bucket = aws_credential['bucket'], Prefix ='resource/')['Contents']
# if size = 0, it's the folder resource/ itself
[d['Key'] for d in content_list1 if d['Size']>0] 

['resource/country.txt']

In [6]:
# call it _bytes because the type of the object retrieved is "bytes"
country_code_bytes=s3.get_object(
    Bucket = aws_credential['bucket'],
    Key = 'resource/' + 'country.txt')['Body'].read()

In [19]:
# the header and the iloc selections are specifically for this table
country_code_raw=pd.read_csv(io.BytesIO(country_code_bytes), delimiter = "\t", header = 3).iloc[1:241]
country_code_raw.head(2)

,Code | Name | ISO Code
1,1000 | United States of America ...
2,1010 | Greenland ...


In [29]:
print(country_code_raw.columns)
new_cols_country_code=[col.strip().split(' ')[0].lower() for col in country_code_raw.columns[0].split('|')]
new_cols_country_code

Index(['Code     |    Name                                                | ISO Code'], dtype='object')


['code', 'name', 'iso']

In [48]:
# data frame from NESTED LIST(series) COMPREHENSION
country_code=pd.DataFrame(data = [[e.strip() for e in row] # remove excessive spaces in every element in a row
                     for row in country_code_raw.iloc[:,0].str.split('|')], # split each row by | into columns
             columns=new_cols_country_code)
country_code.head(2)

,code,name,iso
0,1000,United States of America,US
1,1010,Greenland,GL


## Messy country label data

In [49]:
country_messy_bytes=s3.get_object(
    Bucket = aws_credential['bucket'],
    Key = 'visa_output/' + 'country_list.txt')['Body'].read()
print(type(country_messy_bytes))

<class 'bytes'>


In [54]:
country_messy=pd.read_csv(io.BytesIO(country_messy_bytes), delimiter = "\t", names = ['country'])
country_messy.head(2)

,country
0,COTE D'IVOIRE
1,GREAT BRITAIN AND NORTHERN IRELAND


# Record linkage

## TfidfVectorizer

In [56]:
tfidf_vectorizer = TfidfVectorizer(stop_words = 'english')
corpus = tfidf_vectorizer.fit_transform(country_code['name'])
type(corpus)

scipy.sparse._csr.csr_matrix

In [70]:
print(f'corpus.shape: {corpus.shape}')
corpus.toarray()

corpus.shape: (240, 298)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [63]:
print(f"""The type of `tfidf_vectorizer.get_feature_names_out()` is \
{type(tfidf_vectorizer.get_feature_names_out())}. 
The length of `tfidf_vectorizer.get_feature_names_out()` is \
{len(tfidf_vectorizer.get_feature_names_out())}.
First few items in it are: {tfidf_vectorizer.get_feature_names_out()[:20]}""")

The type of `tfidf_vectorizer.get_feature_names_out()` is <class 'numpy.ndarray'>. 
The length of `tfidf_vectorizer.get_feature_names_out()` is 298.
First few items in it are: ['administered' 'afghanistan' 'africa' 'african' 'albania' 'algeria'
 'america' 'american' 'andorra' 'angola' 'anguilla' 'antarctic' 'antigua'
 'arab' 'arabia' 'argentina' 'armenia' 'aruba' 'australia' 'austria']


## Tfidf Matrix

In [68]:
# Create a data frame of the tfidf matrix, column names are the features
tfidf_matrix_df = pd.DataFrame(corpus.toarray(),\
                               columns = tfidf_vectorizer.get_feature_names_out())

## Cosine-similarity

In [71]:
n = 2 # Keep top 2 best matches
# transform the messy data to be matched
for i in range(len(country_messy['country'])):
    # loop over every row in the messy data
    query = country_messy['country'].iloc[i]
    # tokenize and vectorize each individual query row, must be wrap with []
    query_vector = tfidf_vectorizer.transform([query])
    # run cosine-similarity, corpus is the fitted and transformed(vectorized) label dictionary data
    cosine_sim = pd.DataFrame(cosine_similarity(corpus, query_vector),\
                             columns = ['cosine_similarity'],\
                             index = country_code.index) # use index of label dictionary data
    cosine_sim = cosine_sim.sort_values(by = ['cosine_similarity'], ascending = False)
    # Keep top n best matches, and keep all columns(`,:]`) in the coded/labeled/dictionary
    # output gives potential matches as country names
    output = country_code.loc[cosine_sim.index[0:n],:]
    output.index = ['match_1','match_2']
    # scores report the cosine similarity score of these matches
    scores = cosine_sim[0:n]
    
    # NOW, CREATE FINAL OUTPUTS
    if i == 0:
        # `deep = True`, a new object will be created with a copy of the calling object’s data and indices
        all_outputs = output.copy(deep = True) # for the first iteration, just copy the result
        all_scores = scores.copy(deep = True)
    else:
        # from the 2nd iteration onwards
        all_outputs = pd.concat([all_outputs,output]) # append new rows as iterations continue
        all_scores = pd.concat([all_scores, scores])

In [74]:
# Convert the indeces to a regular column
all_outputs['rank'] = all_outputs.index
# Must reset index for all dataframes before merging
all_outputs = all_outputs.reset_index(drop = True)
all_outputs.head()

,code,name,iso,rank
0,7480,Cote d'Ivoire,CI,match_1
1,1000,United States of America,US,match_2
2,4190,Ireland,IE,match_1
3,9610,Northern Mariana Islands,MP,match_2
4,7880,Madagascar,MG,match_1


In [75]:
# Must reset index for all dataframes before merging
all_scores = all_scores.reset_index(drop = True)
all_scores.head()

,cosine_similarity
0,1.000000
1,0.000000
2,0.707107
3,0.454054
4,1.000000


In [87]:
# without .values, still creates a data frame but has repetitive index 0,0,1,1,...
messy_repeat_n=pd.DataFrame(np.repeat(country_messy[['country']].values, n, axis = 0),columns=['query'])
messy_repeat_n.tail()

,query
511,SINT MAARTEN
512,SERBIA
513,SERBIA
514,SRI LANKA
515,SRI LANKA


In [109]:
# combine columns from 3 data frames:
# (1)messy labels * n, (2)matched results, and (3)cosine similarity values
all_matches=pd.concat(
    [messy_repeat_n,all_outputs,all_scores],axis=1).\
rename(columns={'name':'match'}).\
sort_values(by=['cosine_similarity','query'], ascending=[False, True]).\
reset_index(drop=True)
# filter values
all_matches = all_matches.loc[
    (all_matches['rank'] == 'match_1')&(all_matches['cosine_similarity']>0.5)].\
reset_index(drop=True)

In [110]:
all_matches

,query,code,match,iso,rank,cosine_similarity
0,ANTIGUA AND BARBUDA,2484,Antigua and Barbuda,AG,match_1,1.000000
1,BOSNIA AND HERZEGOVINA,4793,Bosnia and Herzegovina,BA,match_1,1.000000
2,BOSNIA HERZEGOVINA,4793,Bosnia and Herzegovina,BA,match_1,1.000000
3,BRITISH VIRGIN ISLANDS,2482,British Virgin Islands,VG,match_1,1.000000
4,BURKINA FASO,7600,Burkina Faso,BF,match_1,1.000000
...,...,...,...,...,...,...
240,SAINT MARTIN,2487,Saint Lucia,LC,match_1,0.629568
241,MICRONESIA,6820,"Micronesia, Federated States of",FM,match_1,0.607612
242,CZEC REPUBLIC,4351,Czech Republic,CZ,match_1,0.553170
243,SYRIA,5020,Syria (Syrian Arab Republic),SY,match_1,0.549999
